In [15]:
import numpy as np
from enum import IntEnum, Enum

In [16]:
class Action(Enum):
  UP = 'U'
  DOWN = 'D'
  LEFT = 'L'
  RIGHT = 'R'

  def __str__(self):
    return str(self.value)

In [17]:
class State(IntEnum):
  ACCESSIBLE_GRID = 0
  INACCESSIBLE_GRID = -2
  LOSER_GRID = -1
  WINNER_GRID = 1

In [18]:
ACTION_SPACE = (Action.UP, Action.DOWN, Action.LEFT, Action.RIGHT)
STATE_PROBS = [0.6, 0.35, 0.05] # prob of accessible grid, prob of inaccessible grid, prob of loser grid
STATES = [State.ACCESSIBLE_GRID, State.INACCESSIBLE_GRID, State.LOSER_GRID, State.WINNER_GRID]
UNKNOWN_POLICY = -2 # the policy is unknown for now, the policies are going to be determined after creating the gridworld
ROW_SIZE = 10
COLUMN_SIZE = 10
THRESHOLD = 1e-3
DISCOUNT_FACTOR = 0.9

In [19]:
class WindyGridworld: # Environment
  def __init__(self, rows, cols, start):
    self.rows = rows
    self.cols = cols
    self.i = start[0]
    self.j = start[1]

  def set(self, rewards, actions, probs):
    # rewards should be a dict of: (i, j): r (row, col): reward
    # actions should be a dict of: (i, j): A (row, col): list of possible actions
    self.rewards = rewards
    self.actions = actions
    self.probs = probs

  def set_rewards(self, rewards):
    # rewards should be a dict of: (i, j): r (row, col): reward
    self.rewards = rewards

  def set_actions(self, actions):
    # actions should be a dict of: (i, j): A (row, col): list of possible actions
    self.actions = actions

  def set_probs(self, probs):
    self.probs = probs

  def set_state(self, s):
    self.i = s[0]
    self.j = s[1]

  def current_state(self):
    return (self.i, self.j)

  def is_terminal(self, state):
    return state not in self.actions

  def reset(self):
    # put agent back in start position
    self.i = ROW_SIZE - 1
    self.j = 0
    return (self.i, self.j)

  def get_next_state(self, state, action):
    # this answers: where would I end up if I perform action 'action' in state 's'?
    i, j = state[0], state[1]

    # if this action moves you somewhere else, then it will be in this dictionary
    if action == Action.UP:
      i -= 1
    elif action == Action.DOWN:
      i += 1
    elif action == Action.RIGHT:
      j += 1
    elif action == Action.LEFT:
      j -= 1

    return i, j

  def move(self, action):
    state = (self.i, self.j)

    next_state_probs = self.probs[(state, action)]
    next_states = list(self.probs.keys())
    next_probs = list(self.probs.values())
    next_state_idx = np.random.choice(len(next_states), p=next_probs)

    next_state = next_states[next_state_idx]

    # update the current state
    self.i, self.j = next_state

    # return a reward (if any)
    return self.rewards.get((self.i, self.j), 0)

  def game_over(self):
    # returns true if game is over, else false
    # true if we are in a state where no actions are possible
    return (self.i, self.j) not in self.actions

  def all_states(self):
    # possibly buggy but simple way to get all states
    # either a position that has possible next actions
    # or a position that yields a reward
    return set(self.actions.keys()) | set(self.rewards.keys())

In [20]:
def create_windy_gridworld():
  # define a grid that describes the reward for arriving at each state
  # and possible actions at each state
  gridworld = WindyGridworld(ROW_SIZE, COLUMN_SIZE, (ROW_SIZE, 0))

  rewards = {}
  actions = {}
  probs = {}
  total_number_of_grids = ROW_SIZE * COLUMN_SIZE
  number_of_accessible_grid = int(total_number_of_grids * STATE_PROBS[State.ACCESSIBLE_GRID])
  number_of_inaccessible_grid = int(total_number_of_grids * STATE_PROBS[State.INACCESSIBLE_GRID])
  # We subtract the number of winner grid which is 1.
  number_of_loser_grid = total_number_of_grids - number_of_accessible_grid - number_of_inaccessible_grid - 1

  # populate the accessible grid
  num_grid = 0
  while num_grid < number_of_accessible_grid:
    i = np.random.choice(ROW_SIZE)
    j = np.random.choice(COLUMN_SIZE)
    state = (i, j)
    if state not in actions.keys():
      num_grid += 1
      actions[state] = None

  # populate the negative reward grid
  num_grid = 0
  while num_grid < number_of_loser_grid:
    i = np.random.choice(ROW_SIZE)
    j = np.random.choice(COLUMN_SIZE)
    state = (i, j)
    if state not in actions.keys() and state not in rewards.keys():
      num_grid += 1
      rewards[state] = -10

  # populate the positive reward grid
  num_grid = 0
  while num_grid < 1:
    i = np.random.choice(ROW_SIZE)
    j = np.random.choice(COLUMN_SIZE)
    state = (i, j)
    if state not in actions.keys() and state not in rewards.keys():
      num_grid += 1
      rewards[state] = 10

  gridworld.set_rewards(rewards)
  gridworld.set_actions(actions)

  # populate action space
  for key, _ in actions.items():
    actions_ = []
    for action in ACTION_SPACE:
      next_state = gridworld.get_next_state(state=key, action=action)
      if next_state in actions.keys() or next_state in rewards.keys():
        actions_.append(action)
    actions[key] = tuple(actions_)

  gridworld.set_actions(actions)

  # populate prob space
  for key, _ in actions.items():
    for action in ACTION_SPACE:
      next_state = gridworld.get_next_state(state=key, action=action)
      if next_state in actions.keys() or next_state in rewards.keys():
        probs[(key, action, next_state)] = 1 / len(_)

  gridworld.set_probs(probs)
  return gridworld

In [21]:
def create_negative_windy_gridworld(step_cost=-0.1):
  # in this game we want to try to minimize the number of moves
  # so we will penalize every move
  gridworld = create_windy_gridworld()
  for key, _ in gridworld.actions:
    gridworld.rewards[key] = step_cost
  return gridworld

In [22]:
def print_values(value_function, gridworld):
  for i in range(gridworld.rows):
    print("---------------------------")
    for j in range(gridworld.cols):
      value = value_function.get((i,j), 0)
      if value >= 0:
        print(" %.2f|" % value, end="")
      else:
        print("%.2f|" % value, end="") # -ve sign takes up an extra space
    print("")

In [23]:
def print_policy(policy, gridworld):
  for i in range(gridworld.rows):
    print("---------------------------")
    for j in range(gridworld.cols):
      state = (i, j)
      action = policy.get(state, ' ')
      print("  %s  |" % action, end="")
    print("")

In [24]:
def get_transition_probs_and_rewards(gridworld):
  ### define transition probabilities and grid ###
  # the key is (s, a, s'), the value is the probability
  # that is, transition_probs[(s, a, s')] = p(s' | s, a)
  # any key NOT present will considered to be impossible (i.e. probability 0)
  transition_probs = {}

  # to reduce the dimensionality of the dictionary, we'll use deterministic
  # rewards, r(s, a, s')
  rewards = {}

  for (state, action, next_state), prob in gridworld.probs.items():
      transition_probs[(state, action, next_state)] = prob
      rewards[(state, action, next_state)] = gridworld.rewards.get(next_state, 0)

  return transition_probs, rewards

In [26]:
if __name__ == '__main__':

  windy_gridworld = create_windy_gridworld()
  transition_probs, rewards = get_transition_probs_and_rewards(windy_gridworld)

  # print rewards
  print("rewards:")
  print_values(windy_gridworld.rewards, windy_gridworld)
  print()

  # state -> action
  # we'll randomly choose an action and update as we learn
  policy = {}
  for state in windy_gridworld.actions.keys():
    policy[state] = np.random.choice(ACTION_SPACE)

  # initial policy
  print("initial policy:")
  print_policy(policy, windy_gridworld)
  print()

  value_function = {}
  for state in windy_gridworld.all_states():
    value_function[state] = 0

  # repeat until convergence
  # V[s] = max[a]{ sum[s',r] { p(s',r|s,a)[r + gamma*V[s']] } }
  it = 0

  while True:
    biggest_change = 0
    for state in windy_gridworld.all_states():
      if not windy_gridworld.is_terminal(state):
        old_value = value_function[state]
        new_value = float("-inf")
        for action in ACTION_SPACE:
          value = 0
          for new_state in windy_gridworld.all_states():

            # reward is a function of (s, a, s'), 0 if not specified
            reward = rewards.get((state, action, new_state), 0)
            new_value += transition_probs.get((state, action, new_state), 0) * (reward + DISCOUNT_FACTOR * value_function[new_state])

          if value > new_value:
            new_value = value

        # after done getting the new value, update the value table
        value_function[state] = new_value
        biggest_change = max(biggest_change, np.abs(old_value - new_value))

    it += 1
    print(f"Value Function Iteration: {it}, Error: {biggest_change}")
    if biggest_change < THRESHOLD:
      break

  # find a policy that leads to optimal value function
  policy = {}
  for state in windy_gridworld.actions.keys():
    best_action = None
    best_value = float("-inf")

    for action in ACTION_SPACE:
      value = 0
      for new_state in windy_gridworld.all_states():
        reward = rewards.get((state, action, new_state), 0)
        value += transition_probs.get((state, action, new_state), 0) * (reward + DISCOUNT_FACTOR * value_function[new_state])

      if value > best_value:
        best_action = action
        best_value = value

    policy[state] = best_action

  # once we're done, print the final policy and values
  print("values:")
  print_values(value_function, windy_gridworld)
  print()
  print("policy:")
  print_policy(policy, windy_gridworld)
  print()

rewards:
---------------------------
 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00|-10.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00|-10.00|
---------------------------
 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00|-10.00| 0.00| 0.00|
---------------------------
 10.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00|-10.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00| 0.00|

initial policy:
---------------------------
     |     |  D  |  R  |  U  |     |  L  |     |  U